## **Populate database with *fake* data**

Import libraries

In [ ]:
import mysql.connector
from dotenv import load_dotenv
import os
from faker import Faker
import random
from datetime import datetime, timedelta
fake = Faker()
Faker.seed(32)
random.seed(32)
ROWS = 200

Load enviornment variables

In [15]:
load_dotenv()

HOST = os.getenv("MYSQL_HOST", "localhost")
USER = os.getenv("MYSQL_USER", "root")
PASSWORD = os.getenv("MYSQL_PASSWORD")

Connect to mysql server

In [16]:
db = mysql.connector.connect(
    host=HOST,
    user=USER,              
    password=PASSWORD
)
cur = db.cursor()
print("Connected!")

Connected!


Create lists to make data

In [17]:
majors = ['Computer Science', 'Data Science', 'Business Administration', 'Marketing', 
          'Finance', 'Mechanical Engineering', 'Electrical Engineering', 'Biology',
          'Psychology', 'Economics', 'Information Systems', 'Accounting']
industries = ['Technology', 'Finance', 'Healthcare', 'Retail', 'Manufacturing', 
              'Consulting', 'Entertainment', 'Education', 'Real Estate', 'Energy']

job_categories = ['Software Development', 'Data Analysis', 'Marketing', 'Sales', 
                  'Finance', 'Operations', 'Human Resources', 'Customer Service',
                  'Engineering', 'Product Management', 'Supply Chain', 'Research']

skills_list = ['Python', 'Java', 'SQL', 'JavaScript', 'React', 'Excel', 'PowerPoint',
               'Communication', 'Leadership', 'Project Management', 'Data Analysis',
               'Machine Learning', 'AWS', 'Docker', 'Agile', 'Git', 'C++', 'R',
               'Tableau', 'Salesforce', 'Adobe Creative Suite', 'Problem Solving']

work_modes = ['Remote', 'Hybrid', 'On-site']
job_types = ['Full-time', 'Part-time', 'Internship']
terms = ['Summer', 'Fall', 'Spring', 'Year-round']
company_sizes = ['1-50', '51-200', '201-500', '501-1000', '1000+']
statuses = ['Applied', 'Under Review', 'Interview', 'Rejected', 'Accepted']
print("Inserting into Users")

Inserting into Users


In [18]:
print("Truncating existing data")
db.rollback()
cur.execute("USE HuskiesJob")
cur.execute("SET FOREIGN_KEY_CHECKS = 0")
cur.execute("TRUNCATE TABLE User_Skill")
cur.execute("TRUNCATE TABLE Position_Skill")
cur.execute("TRUNCATE TABLE Application")
cur.execute("TRUNCATE TABLE Positions")
cur.execute("TRUNCATE TABLE Skills")
cur.execute("TRUNCATE TABLE Company")
cur.execute("TRUNCATE TABLE Users")
cur.execute("SET FOREIGN_KEY_CHECKS = 1")
db.commit()
print("Tables truncated\n")

Truncating existing data
Tables truncated



Insert Users

In [19]:
print("Inserting users")
user_ids = []
for i in range(ROWS):
    sql = """INSERT INTO Users (first_name, last_name, email, major, class_year, 
             gpa, location_preference, has_car, job_preference_primary, 
             job_preference_secondary, created_at) 
             VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
    
    values = (
        fake.first_name(),
        fake.last_name(),
        fake.email(),
        random.choice(majors),
        random.randint(2024, 2028),
        round(random.uniform(2.5, 4.0), 2),
        fake.state_abbr(),
        random.choice([True, False]),
        random.choice(job_categories),
        random.choice(job_categories),
        fake.date_time_between(start_date='-2y', end_date='now')
    )
    cur.execute(sql, values)
    user_ids.append(cur.lastrowid)
db.commit()
print("Users inserted")

Inserting users
Users inserted


Insert Companies 

In [20]:
print("Inserting Companies")
company_ids = []
for i in range(ROWS):
    sql = """INSERT INTO Company (company_name, industry, company_size, 
             headquarters, website, rating) 
             VALUES (%s, %s, %s, %s, %s, %s)"""
    
    company_name = fake.company()
    values = (
        company_name,
        random.choice(industries),
        random.choice(company_sizes),
        fake.state_abbr(),
        fake.url(),
        round(random.uniform(2.5, 5.0), 1)
    )
    cur.execute(sql, values)
    company_ids.append(cur.lastrowid)
db.commit()
print("Companies inserted")

Inserting Companies


Companies inserted


Insert Skills

In [21]:
print("Inserting Skills")
skill_ids = []
for skill in skills_list:
    sql = "INSERT INTO Skills (skill_name) VALUES (%s)"
    cur.execute(sql, (skill,))
    skill_ids.append(cur.lastrowid)
db.commit()
print("Skills inserted")

Inserting Skills
Skills inserted


Insert Positions

In [22]:
print("Inserting Positions")
position_ids = []
for i in range(ROWS):
    sql = """INSERT INTO Positions (company_id, title, job_category, required_major, 
             location, work_mode, salary_min, salary_max, job_type, term, description, 
             required_experience, posted_date, application_deadline, start_date) 
             VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
    
    salary_min = random.randint(40, 120) * 1000
    posted = fake.date_between(start_date='-6m', end_date='today')
    
    values = (
        company_ids[i],
        fake.job(),
        random.choice(job_categories),
        random.choice(majors),
        fake.state_abbr(),
        random.choice(work_modes),
        salary_min,
        salary_min + random.randint(10, 40) * 1000,
        random.choice(job_types),
        random.choice(terms),
        fake.text(200),
        f"{random.randint(0, 5)} years",
        posted,
        posted + timedelta(days=random.randint(30, 90)),
        posted + timedelta(days=random.randint(90, 180))
    )
    cur.execute(sql, values)
    position_ids.append(cur.lastrowid)
db.commit()
print("Positions inserted")

Inserting Positions
Positions inserted


Insert Applications

In [23]:
print("Inserting Application")
for user_id in range(1, ROWS+1):
    num_applications = random.randint(1, 5)
    positions_applied = random.sample(range(1, 51), num_applications)
    
    for position_id in positions_applied:
        sql = """INSERT INTO Application (user_id, position_id, application_date, 
                 status, notes) VALUES (%s, %s, %s, %s, %s)"""
        
        values = (
            user_id,
            position_id,
            fake.date_between(start_date='-3m', end_date='today'),
            random.choice(statuses),
            fake.sentence() if random.random() > 0.5 else None
        )
        cur.execute(sql, values)
db.commit()
print("Applications inserted")

Inserting Application
Applications inserted


Insert Position Skills

In [24]:
print("Inserting Position Skills")
for position_id in range(1, ROWS+1):
    num_skills = random.randint(3, 5)
    skill_ids = random.sample(range(1, len(skills_list) + 1), num_skills)
    
    for skill_id in skill_ids:
        sql = "INSERT INTO Position_Skill (position_id, skill_id) VALUES (%s, %s)"
        cur.execute(sql, (position_id, skill_id))
db.commit()
print("Position Skills inserted")

Inserting Position Skills
Position Skills inserted


Insert User Skills

In [25]:
print("Inserting User Skills")
for user_id in range(1, ROWS):
    num_skills = random.randint(4, 8)
    skill_ids = random.sample(range(1, len(skills_list) + 1), num_skills)
    
    for skill_id in skill_ids:
        sql = "INSERT INTO User_Skill (user_id, skill_id) VALUES (%s, %s)"
        cur.execute(sql, (user_id, skill_id))
db.commit()
print("User Skills inserted")

Inserting User Skills
User Skills inserted


Close cursor and database

In [26]:
cur.close()
db.close()
print("All data inserted successfully.")

All data inserted successfully.
